# DuckDB in Jupyter Notebooks

We can also run DuckDB in Python for an easier scripting environment.

In [1]:
# Be sure to first `!pip install duckdb`
import duckdb

In [2]:
# Set up / Reconnect to a local DuckDB database
conn = duckdb.connect("tmp.dbb");

# Set up Spatial & H3 Extensions
conn.sql("""
    INSTALL spatial;
    INSTALL h3 from community;
    LOAD spatial;
    LOAD h3;
""")

In [39]:
df = conn.sql(f"""
    SELECT
        id,
        names.primary as name,
        categories.primary as category,
        confidence,
        ST_ASText(geometry) as wkt
    FROM read_parquet('s3://overturemaps-us-west-2/release/2025-04-23.0/theme=places/type=place/*')
    WHERE
        bbox.xmin BETWEEN -112.101 AND -111.740
        AND bbox.ymin BETWEEN 40.699 AND 40.853
""").df()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [40]:
df

,id,name,category,confidence,wkt
0,08f269602b36dd83031c482287a964f3,Pleasant Green Park,park,0.978172,POINT (-112.0945883 40.70094)
1,08f269602baec0440318ca7ac620e59a,Magna Elementary School,elementary_school,0.978172,POINT (-112.0948915 40.7039208)
2,08f2696076ca0cb103196ea2d3cb51fd,Magna Recreation Center,sports_and_recreation_venue,0.978172,POINT (-112.0922502 40.700182)
3,08f2696076d9cc5203445cb3f7bdb886,Magna Outdoor Pool,swimming_pool,0.978172,POINT (-112.0933876 40.7013161)
4,08f269602ba5d8900352bf0296a1383c,The Church of Jesus Christ of Latter-day Saints,church_cathedral,0.978172,POINT (-112.0923506 40.7043022)
...,...,...,...,...,...
16384,08f2696b91aa04c80307833d832988dd,Don M. Stomquist House,landmark_and_historical_building,0.472681,POINT (-111.8566838 40.8481366)
16385,08f2696b8375d1b4036df01d7cc40794,Rudys Flat,hiking_trail,0.213371,POINT (-111.8118439 40.8394094)
16386,08f2696b8264656b0300327819cc3fa3,Cave Peak,cave,0.893631,POINT (-111.8349303 40.8499444)
16387,08f2696aa4819b00030a748af8df37e4,Red Butte Canyon Research Natural Area,park,0.925951,POINT (-111.783179 40.797453)


In [41]:
import geopandas as gpd

In [42]:
# Create Geometries from WKT
geometry = gpd.GeoSeries.from_wkt(df.wkt)
# Drop WKT column
df.drop(columns=["wkt"], inplace=True)

In [43]:
gdf = gpd.GeoDataFrame(df,geometry=geometry)
gdf

,id,name,category,confidence,geometry
0,08f269602b36dd83031c482287a964f3,Pleasant Green Park,park,0.978172,POINT (-112.09459 40.70094)
1,08f269602baec0440318ca7ac620e59a,Magna Elementary School,elementary_school,0.978172,POINT (-112.09489 40.70392)
2,08f2696076ca0cb103196ea2d3cb51fd,Magna Recreation Center,sports_and_recreation_venue,0.978172,POINT (-112.09225 40.70018)
3,08f2696076d9cc5203445cb3f7bdb886,Magna Outdoor Pool,swimming_pool,0.978172,POINT (-112.09339 40.70132)
4,08f269602ba5d8900352bf0296a1383c,The Church of Jesus Christ of Latter-day Saints,church_cathedral,0.978172,POINT (-112.09235 40.7043)
...,...,...,...,...,...
16384,08f2696b91aa04c80307833d832988dd,Don M. Stomquist House,landmark_and_historical_building,0.472681,POINT (-111.85668 40.84814)
16385,08f2696b8375d1b4036df01d7cc40794,Rudys Flat,hiking_trail,0.213371,POINT (-111.81184 40.83941)
16386,08f2696b8264656b0300327819cc3fa3,Cave Peak,cave,0.893631,POINT (-111.83493 40.84994)
16387,08f2696aa4819b00030a748af8df37e4,Red Butte Canyon Research Natural Area,park,0.925951,POINT (-111.78318 40.79745)


In [61]:
gb_categories = (
    gdf.groupby('category')
       .aggregate(
           {'id':['count'],
            'confidence': ['min','mean','max']
           })
)
gb_categories.sort_values(by=('id','count'), ascending=False).head(25)

id confidence                    
                                 count        min      mean       max
category                                                             
landmark_and_historical_building   343   0.075472  0.844299  0.996163
professional_services              319   0.075472  0.747716  0.994980
automotive_repair                  276   0.199005  0.853161  0.996163
church_cathedral                   267   0.075472  0.779183  0.994980
community_services_non_profits     230   0.075472  0.695823  0.994980
contractor                         221   0.199005  0.809601  0.996163
real_estate_agent                  201   0.199005  0.687654  0.996163
beauty_salon                       192   0.199005  0.715161  0.996163
college_university                 170   0.199005  0.762372  0.983319
hospital                           154   0.213371  0.821873  0.996163
freight_and_cargo_service          149   0.199005  0.777107  0.996163
park                               149   0.213371  0.757236  0.996163
car_dealer                         148   0.199005  0.818362  0.996163
doctor                             144   0.213371  0.765634  0.996163
fast_food_restaurant               144   0.213371  0.887051  0.996163
industrial_equipment               143   0.199005  0.812313  0.996163
mexican_restaurant                 141   0.075472  0.887599  0.996163
hotel                              129   0.199005  0.850840  0.996163
coffee_shop                        127   0.213371  0.889132  0.994980
real_estate                        118   0.199005  0.667822  0.996163
atms                               113   0.350515  0.827194  0.994980
tattoo_and_piercing                111   0.199005  0.818648  0.996163
gas_station                        110   0.291372  0.863059  0.996163
lawyer                             109   0.199005  0.794556  0.994980
printing_services                  107   0.199005  0.818347  0.996163

In [36]:
places_per_h3_cell = conn.sql("""
    SELECT
        h3_latlng_to_cell_string(bbox.ymin, bbox.xmin, 9) as h3,
        count(1) AS places
    FROM read_parquet('s3://overturemaps-us-west-2/release/2025-04-23.0/theme=places/type=place/*')
    WHERE
        bbox.xmin BETWEEN -112.101 AND -111.740
        AND bbox.ymin BETWEEN 40.699 AND 40.853
        AND confidence > 0.7
    GROUP BY 1
""").df()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [37]:
places_per_h3_cell.places.sum()

np.int64(13207)

In [38]:
len(places_per_h3_cell)

1534